We consider a three-layer transportation network with components that consists of plants
(P), warehouses (W), and customers (C). This system is broken down into two stages of
transportation:
• From plants to warehouses;
• From warehouses to customers.

Part (a): Multi-Source Fulfillment
Customers can receive their demand from multiple warehouses. The constraints ensure
supply, capacity, and demand are respected.

In [ ]:

from gurobipy import Model, GRB, quicksum

INPUT_FILE = "instance1-1.txt"
sep = ","
P, W, C = None, None, None

if __name__ == "__main__":
    with open(INPUT_FILE) as f:
        P, W, C = map(int, f.readline().split(sep))

        # Sets
        plants = range(P)
        warehouses = range(W)
        customers = range(C)

        supply = list(map(int,f.readline().split(sep)))
        capacity = list(map(int, f.readline().split(sep)))
        demand = list(map(int, f.readline().split(sep)))

        # Cost matrices
        a = []
        for _ in range(P):
            # cost from plant i to warehouse j
            a_row = list(map(int, f.readline().split(sep)))
            a.append(a_row)


        b = []
        for _ in range(W):
            # cost from warehouse j to customer k
            b_row = list(map(int, f.readline().split(sep)))
            b.append(b_row)

    if C is None:
        sys.exit()
    # Model
    model = Model("TransportationNetwork")

    # Variables
    x = model.addVars(plants, warehouses, name="x", lb=0)  # Plant -> Warehouse
    y = model.addVars(warehouses, customers, name="y", lb=0)  # Warehouse -> Customer



    # Objective: minimize total cost
    model.setObjective(
        quicksum(a[i][j] * x[i, j] for i in plants for j in warehouses) +
        quicksum(b[j][k] * y[j, k] for j in warehouses for k in customers),
        GRB.MINIMIZE
    )

    # Constraints
    for i in plants:
        model.addConstr(quicksum(x[i, j] for j in warehouses) <= supply[i])

    for j in warehouses:
        model.addConstr(quicksum(y[j, k] for k in customers) <= capacity[j])

    for k in customers:
        model.addConstr(quicksum(y[j, k] for j in warehouses) >= demand[k])

    for j in warehouses:
        model.addConstr(quicksum(x[i, j] for i in plants) >= quicksum(y[j, k] for k in customers))

    # Solve
    model.optimize()

    # Print solution
    if model.status == GRB.OPTIMAL:
        print("Objective value:", model.ObjVal)
        for i in plants:
            for j in warehouses:
                if x[i, j].X > 0:
                    print(f"Ship {x[i,j].X} from Plant {i} to Warehouse {j} \\\\")
        for j in warehouses:
            for k in customers:
                if y[j, k].X > 0:
                    print(f"Ship {y[j,k].X} from Warehouse {j} to Customer {k} \\\\ ")



Part (b): Single-Warehouse Assignment
Each customer must be assigned to only one warehouse. This adds a binary decision
variable and changes the model slightly

In [ ]:
import sys

from gurobipy import Model, GRB, quicksum

INPUT_FILE = "instance1-2.txt"
sep = ","
P, W, C = None, None, None

if __name__ == "__main__":
    with open(INPUT_FILE) as f:
        P, W, C = map(int, f.readline().split(sep))

        # Sets
        plants = range(P)
        warehouses = range(W)
        customers = range(C)

        supply = list(map(int,f.readline().split(sep)))
        capacity = list(map(int, f.readline().split(sep)))
        demand = list(map(int, f.readline().split(sep)))

        # Cost matrices
        a = []
        for _ in range(P):
            # cost from plant i to warehouse j
            a_row = list(map(int, f.readline().split(sep)))
            a.append(a_row)


        b = []
        for _ in range(W):
            # cost from warehouse j to customer k
            b_row = list(map(int, f.readline().split(sep)))
            b.append(b_row)

    if C is None:
        sys.exit()
    # Model
    model = Model("TransportationNetwork")

    # Variables
    x = model.addVars(plants, warehouses, name="x", lb=0)  # Plant -> Warehouse
    y = model.addVars(warehouses, customers, name="y", lb=0)  # Warehouse -> Customer
    z = model.addVars(customers,warehouses, vtype=GRB.INTEGER, name="z", lb=0,ub=1)  # Customer -> Warehouse


    # Objective: minimize total cost
    model.setObjective(
        quicksum(a[i][j] * x[i, j] for i in plants for j in warehouses) +
        quicksum(b[j][k] * y[j, k] for j in warehouses for k in customers),
        GRB.MINIMIZE
    )

    # Constraints
    for i in plants:
        model.addConstr(quicksum(x[i, j] for j in warehouses) <= supply[i])

    for j in warehouses:
        model.addConstr(quicksum(y[j, k] for k in customers) <= capacity[j])

    for k in customers:
        model.addConstr(quicksum(y[j, k] for j in warehouses) >= demand[k])

    for j in warehouses:
        model.addConstr(quicksum(x[i, j] for i in plants) >= quicksum(y[j, k] for k in customers))

    for k in customers:
        model.addConstr(quicksum(z[k, j] for j in warehouses) <= 1)
        model.addConstr(quicksum(z[k, j] for j in warehouses) >= 1)


    for k in customers:
        for j in warehouses:
            model.addConstr(y[j,k] <= demand[k]*z[k,j])




    # Solve
    model.optimize()

    # Print solution
    if model.status == GRB.OPTIMAL:
        print("Objective value:", model.ObjVal)
        for i in plants:
            for j in warehouses:
                if x[i, j].X > 0:
                    print(f"Ship {x[i,j].X} from Plant {i} to Warehouse {j} \\\\")
        for j in warehouses:
            for k in customers:
                if y[j, k].X > 0:
                    print(f"Ship {y[j,k].X} from Warehouse {j} to Customer {k} \\\\ ")


We need to complete a certain number of jobs, each with a processing time, due date,
and release date. These jobs are executed by machines, and a job cannot start before its
release date. It is not strictly necessary to finish a job by its due date, but a penalty is
applied for each unit of time a job finishes late.
This is the classic scheduling problem. Each job must be assigned to exactly one
machine, and no machine can process more than one job at a time. Jobs are non-
preemptive.

Part (a): Maximize the Number of Scheduled Jobs
Ignore both the penalty and the due date – only the release date matters. The goal is to
maximize the number of jobs that can be scheduled while respecting their release dates.
So in this particular formulation we have to achieve the minimal date when all the
jobs are done

In [ ]:
from gurobipy import Model, GRB, quicksum


INPUT_FILE = "instance2-1.txt"
sep = " "

with open(INPUT_FILE) as f:
    n = int(f.readline())
    m = int(f.readline())

    # p, r,d w
    p = [0 for _ in range(n)]
    r = [0 for _ in range(n)]
    d = [0 for _ in range(n)]
    w = [0 for _ in range(n)]

    for i in range(n):
        p[i], r[i], d[i], w[i]  = map(int, f.readline().split())

print(p,r,d,w)
def schedule_jobs_max_completed(n, m, p, r):
    model = Model("MaxScheduledJobs2")

    # Decision variables
    s = model.addVars(n, vtype=GRB.INTEGER, name="start",lb=0)  # start time of job
    a = model.addVars(n, m, vtype=GRB.BINARY, name="assign",lb=0,ub=1)  # job j assigned to machine k
    y = model.addVars(n, n, vtype=GRB.BINARY, name="lesser",lb=0,ub=1)  # job i earlier job j
    T = model.addVars(1, vtype=GRB.INTEGER, name="finish",lb=0)

    # Objective: maximize number of scheduled jobs
    model.setObjective(T[0], GRB.MINIMIZE)

    # Each job can be assigned to at most one machine if it is scheduled
    for j in range(n):
        model.addConstr(quicksum(a[j, k] for k in range(m)) == 1, name=f"assign_job_{j}")


    # Start time must be after release date
    for j in range(n):
        model.addConstr(s[j] >= r[j], name=f"release_{j}")


    # Logical greater /less
    for i in range(n):
        for j in range(i+1,n):
            model.addConstr(y[i,j] + y[j,i] == 1, name=f"lesser1_{i}_{j}")

    # Less condition
    for i in range(n):
        for j in range(n):
            if i==j:
                continue
            model.addConstr(y[i,j] * (s[j] -s[i])>=0, name=f"lesser2_{i}_{j}")

    # Finish
    for i in range(n):
        model.addConstr(T[0] >= s[i]+p[i], name=f"fin_{i}")


    # No overlapping on same machine
    M = 2*sum(p)
    for i in range(n):
        for j in range(n):
            for k in range(m):
                if j != i:
                    # Enforce disjunctive constraint
                    model.addConstr(
                        s[i] + p[i]<= s[j] + M*(1 -a[i,k]*a[j,k]) + M*(1-y[i,j]), name=f"no_overlap_{j}_{i}_on_{k}"
                    )

    # added constraint on due date:

    model.setParam("OutputFlag", 0)
    model.optimize()

    # Output results
    if model.status == GRB.OPTIMAL:
        scheduled_jobs = []
        # print("gggg")
        for j in range(n):

            # print("s=", s[j].x,":")
            # print(a[j,0].x,a[j,1].x,end="\t\t\t")
            #
            # for i in range(n):
            #     print(y[j,i].x, end=", ")
            #
            # print(j, m)
            # print(a[j, 0].x, a[j, 1].x, end="!!!!")
            assigned_machine = [k for k in range(m) if a[j, k].X > 0.5]
            # print(assigned_machine, end="!!")

            scheduled_jobs.append((j, s[j].X, assigned_machine))
        return scheduled_jobs, T[0].x
    else:
        print(model.status)
        return None


jobs_done, T = schedule_jobs_max_completed(n, m, p, r)
# print(jobs_done)
print(f"Minimal completion time is {T} \\\\ ")
for job in jobs_done:
    print(f"Job {job[0]+1} is starting at {job[1]} on machine {job[2][0]+1} \\\\ ")




Part (b): Minimize Total Weighted Tardiness
We should consider the due dates and penalties. The objective is to minimize the total
penalty across all jobs for finishing them after their due dates

In [ ]:
from gurobipy import Model, GRB, quicksum


INPUT_FILE = "instance2-1.txt"
sep = " "

with open(INPUT_FILE) as f:
    n = int(f.readline())
    m = int(f.readline())

    # p, r,d w
    p = [0 for _ in range(n)]
    r = [0 for _ in range(n)]
    d = [0 for _ in range(n)]
    w = [0 for _ in range(n)]

    for i in range(n):
        p[i], r[i], d[i], w[i]  = map(int, f.readline().split())



def schedule_jobs_min_penalty(n, m, p, r, d, w):
    model = Model("MinimizePenalty")

    # Variables
    s = model.addVars(n, vtype=GRB.INTEGER, name="start")       # start time
    a = model.addVars(n, m, vtype=GRB.BINARY, name="assign")       # machine assignment
    tard = model.addVars(n, vtype=GRB.INTEGER, name="tardiness")  # tardiness (>=0)
    y = model.addVars(n, n, vtype=GRB.BINARY, name="lesser")  # job i earlier job j

    # Objective: Minimize total weighted tardiness
    model.setObjective(quicksum(w[j] * tard[j] for j in range(n)), GRB.MINIMIZE)

    # Assign each job to at most one machine if scheduled
    for j in range(n):
        model.addConstr(quicksum(a[j, i] for i in range(m)) == 1, name=f"assign_{j}")

    # Start time must respect release date
    for j in range(n):
        model.addConstr(s[j] >= r[j], name=f"release_{j}")

    # Logical greater /less
    for i in range(n):
        for j in range(i + 1, n):
            model.addConstr(y[i, j] + y[j, i] == 1, name=f"lesser1_{i}_{j}")

    # Less condition
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            model.addConstr(y[i, j] * (s[j] - s[i]) >= 0, name=f"lesser2_{i}_{j}")

    # No overlapping on same machine
    M = 10*sum(p)
    for i in range(n):
        for j in range(n):
            for k in range(m):
                if j != i:
                    # Enforce disjunctive constraint
                    model.addConstr(
                        s[i] + p[i] <= s[j] + M * (1 - a[i, k] * a[j, k]) + M * (1 - y[i, j]),
                        name=f"no_overlap_{i}_{j}_on_{k}"
                    )


    # Tardiness: max(0, finish - due date)
    for j in range(n):
        model.addConstr(tard[j] >= s[j] + p[j] - d[j], name=f"tard_{j}")
        # model.addConstr(tard[j] >= 0)


    model.setParam("OutputFlag", 0)
    model.optimize()

    # Output results
    if model.status == GRB.OPTIMAL:
        results = []

        for j in range(n):
            assigned_machine = [i for i in range(m) if a[j, i].X > 0.5][0]
            results.append({
                    "job": j+1,
                    "start": s[j].X,
                    "machine": assigned_machine + 1,
                    "tardiness": tard[j].X,
                    "penalty": tard[j].X * w[j]
                })
        return results
    else:
        print("error", model.status)
        return None


jobs = schedule_jobs_min_penalty(n, m, p, r, d, w)
for job in jobs:
    # print(job, end="\\\\ \n")
    print(f" Job {job['job']} started {job['start']} on machine {job['machine']} \\\\ ")

This problem models a car sharing scheduling system. This is a car sharing scheduling
problem. The company wants to serve as many customer requests as possible, minimizing
total penalties. And we are given:
• Start time and duration (in minutes)
• Preferred zone (where the request should be served from)
• Eligible zones or vehicles that can serve it
• Penalty 1 if the request is not served at all
• Penalty 2 if the request is served from a neighboring zone (not the preferred one)

In [ ]:
import gurobipy as gp
from gurobipy import GRB


def parse_custom_csv(content):
    data = {'requests': [], 'zones': {}, 'vehicles': [], 'days': 0}
    current_section = None
    lines = content.strip().split('\n')

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith('+'):
            current_section = line[1:].split(':')[0].strip().lower()
            continue
        if current_section == 'requests':
            fields = line.split(';')
            req = {
                'id': fields[0],
                'zone': fields[1],
                'day': int(fields[2]),
                'start': int(fields[3]),
                'duration': int(fields[4]),
                'vehicles': fields[5].split('\t'),
                'penalty1': int(fields[6]),
                'penalty2': int(fields[7])
            }
            data['requests'].append(req)
        elif current_section == 'zones':
            fields = line.split(';')
            zone = fields[0]
            adjacent = fields[1].split('\t') if len(fields) > 1 else []
            data['zones'][zone] = adjacent
        elif current_section == 'vehicles':
            data['vehicles'].append(line.split(';')[0].strip())
        elif current_section == 'days':
            data['days'] = int(line.strip())
    return data


def solve_with_gurobi(parsed_data):
    model = gp.Model("CarSharing")

    # Create variables
    x = {}
    y = {}

    requests = parsed_data['requests']
    zones = parsed_data['zones']
    vehicles = parsed_data['vehicles']

    for req in requests:
        r = req['id']
        x[r] = model.addVar(vtype=GRB.BINARY, name=f"x_{r}")
        eligible_zones = [req['zone']] + zones[req['zone']]
        for v in req['vehicles']:
            for z in eligible_zones:
                y[(r, v, z)] = model.addVar(vtype=GRB.BINARY, name=f"y_{r}_{v}_{z}")

    # Objective function
    obj = gp.quicksum(
        (1 - x[req['id']]) * req['penalty1'] +
        gp.quicksum(
            y[(req['id'], v, z)] * req['penalty2']
            for v in req['vehicles']
            for z in zones[req['zone']]  # Adjacent zones only
        )
        for req in requests
    )
    model.setObjective(obj, GRB.MINIMIZE)

    # Assignment constraints
    for req in requests:
        r = req['id']
        model.addConstr(
            gp.quicksum(y[(r, v, z)]
                        for v in req['vehicles']
                        for z in [req['zone']] + zones[req['zone']]
                        ) == x[r],
            name=f"assign_{r}"
        )

    # Vehicle conflict constraints (corrected)
    for v in vehicles:
        # Fix: Use correct variable 'v' in list comprehension
        relevant_requests = [req for req in requests if v in req['vehicles']]

        for i in range(len(relevant_requests)):
            req1 = relevant_requests[i]
            eligible_zones1 = [req1['zone']] + zones[req1['zone']]

            for j in range(i + 1, len(relevant_requests)):
                req2 = relevant_requests[j]
                eligible_zones2 = [req2['zone']] + zones[req2['zone']]

                if req1['day'] != req2['day']:
                    continue

                # Check time overlap
                start1 = req1['start']
                end1 = start1 + req1['duration']
                start2 = req2['start']
                end2 = start2 + req2['duration']

                if (start1 < end2) and (start2 < end1):
                    # Corrected: Sum all possible zone assignments
                    model.addConstr(
                        gp.quicksum(y[(req1['id'], v, z)] for z in eligible_zones1) +
                        gp.quicksum(y[(req2['id'], v, z)] for z in eligible_zones2)
                        <= 1,
                        name=f"conflict_{v}_{req1['id']}_{req2['id']}"
                    )

    # Solve and process results
    model.optimize()
    # --- SOLUTION PROCESSING STARTS HERE ---
    results = []
    total_penalty = 0

    if model.status == GRB.OPTIMAL:
        print("\nOptimal solution found!")

        # Process each request
        for req in parsed_data['requests']:
            req_id = req['id']
            served = x[req_id].X > 0.5  # Check if served
            assignment = None
            penalty = 0

            if served:
                # Find vehicle and zone assignment
                for v in req['vehicles']:
                    for z in [req['zone']] + zones[req['zone']]:
                        if y[(req_id, v, z)].X > 0.5:
                            assignment = (v, z)
                            # Check if non-preferred zone
                            if z != req['zone']:
                                penalty = req['penalty2']
                            break
                    if assignment:
                        break
            else:
                # Apply penalty for unserved request
                penalty = req['penalty1']

            total_penalty += penalty
            results.append((req_id, served, assignment))

        # Print detailed results
        print("\nDetailed assignments:")
        for req_id, served, assignment in results:
            if served:
                print(f"{req_id}: Served by {assignment[0]} from zone {assignment[1]}")
            else:
                print(f"{req_id}: NOT SERVED (Penalty: {req['penalty1']})")

        print(f"\nTotal penalty: {total_penalty}")

    else:
        print("No optimal solution found. Status:", model.status)

    return results, total_penalty
#
# # Example data
# csv_content = """+Requests: 10
# req0;z0;0;1072;127;car2	car3	car0	car1	car4;100;20
# req1;z4;1;648;342;car2	car3	car1	car4	car5;100;20
# req2;z4;1;889;166;car3;100;20
# req3;z0;0;885;314;car1;100;20
# req4;z2;0;780;312;car2	car3	car4	car5;100;20
# req5;z0;1;763;265;car2	car3	car0	car1	car4;100;20
# req6;z3;1;922;188;car2;100;20
# req7;z4;0;568;175;car3	car0	car1;100;20
# req8;z1;0;1034;128;car3	car0;100;20
# req9;z4;0;539;335;car3;100;20
# +Zones: 5
# z0;z1
# z1;z0	z2
# z2;z1	z3
# z3;z2	z4
# z4;z3
# +Vehicles: 6
# car0
# car1
# car2
# car3
# car4
# car5
# +Days: 2"""

# Run solution
INPUT_FILE = "instance05.csv" # "toy1.csv"

with open(INPUT_FILE, 'r') as file:
    file_content = file.read()

print(file_content)
parsed_data = parse_custom_csv(file_content)
results, total_penalty = solve_with_gurobi(parsed_data)

# Print results
print("Optimal Assignment:")
for req_id, served, assignment in results:
    if served:
        print(f"{req_id}: Served by {assignment[0]} from {assignment[1]} \\\\")
    else:
        print(f"{req_id}: Not served\\\\")
print(f"\nTotal Penalty: {total_penalty}\\\\")